## 1. 导入需要的库

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## 2. 模拟数据

接着模拟一组数据，假设batch_x是表示成单词id序列的输入，我们希望通过Attention机制，为一个实例中的每次词分配一个概率得分。

In [22]:
import numpy as np

batch_x = torch.LongTensor(np.array([
    [1, 2, 3, 0, 0, 0],
    [1, 2, 3, 4, 5, 6],
    [1, 2, 0, 0, 0, 0]]
))
mask = (batch_x > 0).float()
batch_size = 3
max_len = 6

设置embedding层：

In [17]:
alphabet_size = 7
input_dim = 8
embedding = nn.Embedding(alphabet_size, input_dim, padding_idx=0)
batch_x_embed = embedding(batch_x)
print(batch_x_embed.size())  # size=(3, 6, 8)

torch.Size([3, 6, 8])


## 3. Attention

### 3.1 设置Attention层

In [18]:
attention_layer = nn.Linear(input_dim, 1)

设置该层只是为了模拟attention实现过程，实际任务中，attention的权重会根据loss进行更新。

### 3.2 错误姿势

首先看一下错误的实现方法：

In [28]:
batch_x_embed = batch_x_embed.view(-1, input_dim)
att_score = attention_layer(batch_x_embed).view(batch_size, max_len)
att_score *= mask
att_weight = F.softmax(att_score, dim=1)
print(att_weight)

tensor([[ 0.1241,  0.0665,  0.3636,  0.1486,  0.1486,  0.1486],
        [ 0.1179,  0.0632,  0.3454,  0.2390,  0.1357,  0.0987],
        [ 0.1581,  0.0848,  0.1893,  0.1893,  0.1893,  0.1893]])


我们可以看到，padding处也有一个概率得分，而这些位置的值应该是0；如果直接将这部分置0，那么padding之前部分的概率之和就不再是1，所以这样的实现方式是错误的。

### 3.3 正确姿势

接下来看一下正确的实现方法：

In [32]:
batch_x_embed = batch_x_embed.view(-1, input_dim)
att_score = attention_layer(batch_x_embed).view(batch_size, max_len)
att_score[mask==0] = -np.inf  # e**inf=0
att_weight = F.softmax(att_score, dim=1)
print(att_weight)

tensor([[ 0.2239,  0.1201,  0.6560,  0.0000,  0.0000,  0.0000],
        [ 0.1179,  0.0632,  0.3454,  0.2390,  0.1357,  0.0987],
        [ 0.6510,  0.3490,  0.0000,  0.0000,  0.0000,  0.0000]])


可以看出，padding部分的权重正确地计算为0，而padding之前的部分概率之和为1。